# Self Query Retriever Test Notebook

In [2]:
import os
import shutil
from loguru import logger
from intellitube.utils import ChatManager
from intellitube.vector_store import VectorStoreManager

## Create Vector Store

In [5]:
vs_path = "test_data/qdrant_vector_store/test_vct_store"
vsc_path = os.path.join(vs_path, "collections")

for path in [vsc_path, vs_path]:
    if os.path.exists(path):
        logger.info(f"Removing path: {path}")
        shutil.rmtree(path)

vectorstore = VectorStoreManager(
    path_on_disk=vs_path,
    collection_path_on_disk=vsc_path
)

2025-07-12 13:25:38.599 | INFO     | __main__:<module>:6 - Removing path: test_data/qdrant_vector_store/test_vct_store


OSError: [Errno 39] Directory not empty: 'test_data/qdrant_vector_store/test_vct_store/collection/text-document-rag'